# Setup

In [1]:
# python modules
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import time

In [2]:
import io
from contextlib import redirect_stdout, redirect_stderr
fake_logger = io.StringIO()

In [3]:
from pathlib import Path

In [4]:
import py4cytoscape as p4c
import networkx as nx

In [5]:
import design

In [6]:
base_dir = Path("..")
graph_path = base_dir / "output" / "original-graphs/"
t_graph_path = base_dir / "output" / "threshold-graphs"
input_dir = base_dir / "input" 

## Load calculated thresholds

In [7]:
thresholds_df = pd.read_csv(base_dir / "output" / "thresholds.tsv", sep="\t", index_col=[0])
thresholds_df.head()

,density-minima
contrast,
D-v-C,1.413
H-v-C,1.565
HD-v-C,1.527
HD-v-H,1.440
W-v-C,1.498


## Functions

In [8]:
def read_graph(path, data=None, create_using=nx.MultiDiGraph, 
               add_node_attributes=False, node_attibutes_df=None, node_attibutes_index_col="name", 
               message=True, end="  ",  header=False, corr=True
              ):
    if data == None:
        data = [('weight', float), ('data_source', str), ('treatment', str)]
        
    with open(path, "rb") as handle:
        if header:
            handle.readline()
        g = nx.read_edgelist(handle, delimiter="\t", create_using=create_using, data=data)
        
    if add_node_attributes:
        nx.set_node_attributes(g, node_attibutes_df.set_index(node_attibutes_index_col).to_dict(orient="index"))
    if corr:
        nx.set_edge_attributes(g, {e: "diablo-corr" for e in g.edges(keys=True)}, 'interaction')
    
    if message:
        print(f"{g.number_of_nodes()}, {g.number_of_edges()}", end=end)
    return g

def threshold_graph(G, thr, message=True, end=" "):
    G.remove_edges_from([(s, t) for (s, t, data) in G.edges(data=True) if abs(data["weight"]) < thr])
    G.remove_nodes_from(list(nx.isolates(G)))
    if message:
        print(f"--> {G.number_of_nodes()}, {G.number_of_edges()}  ", end=end)
    nx.set_node_attributes(G, {k: float(v) for k, v in dict(G.degree()).items()}, 'degree')
    return G

def to_cytoscape(G, title="TEST", collection="TEST", layout_alg="cose", relayout=True, copycat=None, style="weight", node_table=None):
    with redirect_stdout(fake_logger), redirect_stderr(fake_logger):
        cy_suid = p4c.networks.create_network_from_networkx(G, title=title, collection=collection)
        print(copycat, 'copcat')
        
        if copycat:
            print(copycat)
            p4c.layouts.layout_copycat(copycat, cy_suid)
        
        if layout_alg and not copycat:
            p4c.layouts.layout_network(layout_alg, network=cy_suid)
        
        p4c.styles.set_visual_style(style, network=cy_suid)         
        
        if node_table is not None:
            p4c.load_table_data(
            node_table, 
            data_key_column='name', 
            table='node', table_key_column='name',
            network=cy_suid
        )
        
        if relayout:
            relyt_suid = p4c.networks.create_subnetwork(nodes='all', 
                                                           nodes_by_col="SUID", 
                                                           subnetwork_name="cose layout", 
                                                           network=cy_suid)
            p4c.layouts.layout_network(layout_alg, network=relyt_suid)
            p4c.styles.set_visual_style(style, network=relyt_suid)
        
        else:
            relyt_suid = None
        
    return cy_suid, relyt_suid


## Node annotations

In [9]:
node_df = pd.read_csv(input_dir / "node-annotations.tsv", sep="\t")
node_df["mapman_bin"] = node_df["mapman_bin"].apply(lambda x: x.split(" | ") if not pd.isna(x) else np.nan)
node_df.tail()

,biochem_net_name,display_name,pathway,protein_name,short_name,name,group,mapman_bin,description
94,VdnPW3_37011,CLPB4,HSP,VdnPW3_37011,CLPB4,pr.CLPB4,proteomics,"[20.2.1 stress.abiotic.heat, 20.2 stress.abiot...",Chaperone protein clpB 2
95,VdnPW4_15664,ACA11,Ca,VdnPW4_15664,ACA11,pr.ACA11,proteomics,"[34.21 transport.calcium, 30.3 signalling.calc...",Calcium-transporting ATPase 1
96,VdnPW4_18801,KARI,Amino,VdnPW4_18801,F14P22.200,pr.F14P22.200,proteomics,[13.1.4.1.2 amino acid metabolism.synthesis.br...,Ketol-acid reductoisomerase
97,VdnPW5_1541,LHCA3,PS,VdnPW5_1541,LHCA3,pr.LHCA3,proteomics,"[1.1.2.1 PS.lightreaction.photosystem I.LHC-I,...","Chlorophyll a-b binding protein 8, chloroplastic"
98,WATER.CONSUMPTION,Water consumption,NaN,NaN,NaN,ph.WATER.CONSUMPTION,phenomics,NaN,Absolute weight trait based on weighing of the...


## Biochem graph

In [10]:
biochem_graph = read_graph(input_dir/ "KnowledgeNetwork-biochem.tsv", data=[("interaction", str)], 
                           add_node_attributes=True, node_attibutes_df=node_df,
                           message=True, header=True, corr=False)
biochem_graph

90, 94  

In [11]:
def create_merge_whole_graph(dataset, thr_type="whole-graph"):
    all_dfs = []

    for contrast in design.contrasts[dataset]:
        print(contrast,  end="\t")
        G = read_graph(graph_path / dataset / f"{dataset}.{contrast}.ncol", data=[('weight', float)], create_using=nx.MultiGraph,
                      add_node_attributes=True, node_attibutes_df=node_df)
        thr = thresholds_df.loc[dataset][thr_type][contrast]

        G = threshold_graph(G, thr, end="\n")

        nx.set_edge_attributes(G, {k: contrast for k in G.edges(keys=True)}, 'contrast')

        df = nx.to_pandas_edgelist(G)
        all_dfs.append(df)

        df.to_csv(t_graph_path / f"{dataset}.{contrast}.whole_graph.{thr}.ncol", sep="\t", index=None)  

    merged = pd.concat(all_dfs)
    display(merged.head())

    merged.to_csv(t_graph_path / f"{dataset}.TvC-combined.whole_graph.ncol", sep="\t", index=None) 

    G = nx.from_pandas_edgelist(merged, create_using=nx.MultiGraph, edge_attr=["contrast", "weight"])

    print(G.number_of_nodes(), G.number_of_edges())
    cy_whole_graph_merged, _ = to_cytoscape(
        G, 
        title=f"Merged (whole-graph)", 
        collection=f"{dataset} Merged (diff correlation)", 
        relayout=False, 
        copycat=None, 
        style="contrast-merged",
        layout_alg="cose"
    )
    
    print("-----------")
    print(fake_logger.getvalue())
    
    return cy_whole_graph_merged

In [12]:
def add_context(G, contrast):
    ''' add original edges within treatment '''
#     print(G.number_of_nodes(), G.number_of_edges())

    treatment, control = design.contrasts[contrast]

    G_control = read_graph(graph_path / f"{control}.ncol", data=[('weight', float)], message=False,
                          add_node_attributes=True, node_attibutes_df=node_df)
    G_control.remove_edges_from([(s, t) for (s, t) in G_control.edges() if not (s, t) in G.edges()])
    G_control.remove_nodes_from(list(nx.isolates(G_control)))
    nx.set_edge_attributes(G_control, {k: "control" for k in G_control.edges(keys=True)}, 'type')
    nx.set_edge_attributes(G_control, {(u, v, k): d["weight"] for u, v, k, d in G_control.edges(data=True, keys=True)}, 'actual weight')
    nx.set_edge_attributes(G_control, {(u, v, k): d["weight"]*2 for u, v, k, d in G_control.edges(data=True, keys=True)}, 'weight')

    G_treatment = read_graph(graph_path / f"{treatment}.ncol", data=[('weight', float)], message=False)
    G_treatment.remove_edges_from([(s, t) for (s, t) in G_treatment.edges() if not (s, t) in G.edges()])
    G_treatment.remove_nodes_from(list(nx.isolates(G_treatment)))
    nx.set_edge_attributes(G_treatment, {k: "treatment" for k in G_treatment.edges(keys=True)}, 'type')
    nx.set_edge_attributes(G_treatment, {(u, v, k): d["weight"] for u, v, k, d in G_treatment.edges(data=True, keys=True)}, 'actual weight')
    nx.set_edge_attributes(G_treatment, {(u, v, k): d["weight"]*2 for u, v, k, d in G_treatment.edges(data=True, keys=True)}, 'weight')

    nx.set_edge_attributes(G, {k: "treatment-control" for k in G.edges(keys=True)}, 'type')
    G.add_edges_from(G_control.edges(data=True))
    G.add_edges_from(G_treatment.edges(data=True))
    
#     print(G.number_of_nodes(), G.number_of_edges())
    return G

In [13]:
style = "diablo-diff-weight"

In [14]:
def load_all_thresholded_networks(prefix=""):
    for contrast in design.contrasts:

        collection = f"{contrast} (diff correlation)"

        G = read_graph(graph_path / f"{prefix}{contrast}.ncol", data=[('weight', float)],
                      add_node_attributes=True, node_attibutes_df=node_df)
        print(contrast, "   \t", G.number_of_nodes(), G.number_of_edges(), end="\t")

        thr = thresholds_df.loc[contrast][thr_type]

        this_G = threshold_graph(G.copy(), thr)

        cy_suid, _ = to_cytoscape(
            this_G,
            title=f"{contrast} ({thr_type} -- {thr})", 
            collection=collection, 
            relayout=False, 
            copycat=False, 
            style=style,
        ) 

        ########################################################

        context_G = add_context(this_G, contrast)
        _ = to_cytoscape(
            context_G,
            title=f"{contrast} ({thr_type} -- {thr}) context", 
            collection=collection, 
            relayout=False, 
            style=style,
            layout_alg="force-directed"
        )         


        B = context_G.copy()
        new_nodes = []
        new_edges = []
        for b, data in biochem_graph.nodes(data=True):
            if not b in context_G.nodes():
                new_nodes.append((b, data))
        B.add_nodes_from(new_nodes)
        for u, v, data in biochem_graph.edges(data=True):
            # if (u in context_G.nodes()) or (v in context_G.nodes()):
            new_edges.append((u, v, data))
        B.add_edges_from(new_edges)
        B.number_of_nodes(), B.number_of_edges()
        nx.set_node_attributes(B, {v:data for v, data in biochem_graph.nodes(data=True)})
        _ = to_cytoscape(
            B,
            title=f"{contrast} ({thr_type} -- {thr}) context+biochem", 
            collection=collection, 
            relayout=False, 
            style=style,
            layout_alg="force-directed"
        ) 
        time.sleep(5)
        print()


In [15]:
thr_type = 'density-minima'

In [16]:
# Open cytoscape first!
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.2',
 'automationAPIVersion': '1.9.0',
 'py4cytoscapeVersion': '1.9.0'}

In [17]:
blank_session_file = base_dir / "input" / f"biochem-net-basis.cys" 

In [18]:
session = p4c.session.open_session(file_location=str(blank_session_file.absolute()))

Opening /home/cbleker/research/NIB/projects/ADAPT/pilot-networks/multiOmics-integration/_p_ADAPTOmics/_I_Desiree/_S_multiOmics/_A_multiOmics-differential-networks-Py/input/biochem-net-basis.cys...


In [19]:
# RUN TO IMPORT BIOCHEM KNOWLEDGE NETWORK
# biochem_cy = to_cytoscape(biochem_graph, title="Biochem graph", collection="Biochem graph", 
#              relayout=False, copycat=None, layout_alg="force-directed")

In [20]:
load_all_thresholded_networks()

86, 2804  D-v-C    	 86 2804	--> 61, 108   
86, 2800  H-v-C    	 86 2800	--> 19, 19   
86, 2796  HD-v-C    	 86 2796	--> 41, 39   
87, 2845  HD-v-H    	 87 2845	--> 32, 32   
86, 2800  W-v-C    	 86 2800	--> 48, 74   


In [21]:
session_file = base_dir / "output" / f"pilot-diablo-differential-networks.cys" 
p4c.session.save_session(str(session_file.absolute()))

This file has been overwritten.


{}